In [9]:
# start up again with our simple example
from databaker.framework import *
tabs = loadxlstabs("example1.xls")
tab = tabs[0]  # there is only one table
#savepreviewhtml(tabs[0])


Loading example1.xls which has size 7168 bytes
Table names: ['beatles', 'stones']


In [2]:
r1 = tab.excel_ref('B3').expand(RIGHT)
r2 = tab.excel_ref('A3').fill(DOWN)
obs = tab.excel_ref('B4').expand(DOWN).expand(RIGHT).is_not_blank().is_not_whitespace()
dimensions = [ 
    HDim(tab.excel_ref('B1'), TIME, CLOSEST, ABOVE), 
    HDim(r1, "Vehicles", DIRECTLY, ABOVE), 
    HDim(r2, "Name", DIRECTLY, LEFT), 
    HDimConst("Category", "Beatles")
]
c1 = ConversionSegment(tab, dimensions, obs)


In [3]:
#writetechnicalCSV(None, c1)

In [12]:
import io, six, csv
def Lyield_dimension_values(values, headernames):
    for k in template.headermeasurements:
        if isinstance(k, tuple):
            yield values.get(template.headermeasurementnumvalues[k[1]], '')
        else:
            yield ''

    for dimension in range(1, len(headernames)):
        name = headernames[dimension]
        value = values[dimension]
        topic_headers = template.get_topic_headers(name, value)
        for col in topic_headers:
            yield col

def Lrewrite_headers(row,dims):
    for i in range(0,len(row)):
        if i >= len(template.start.split(',')):
            which_cell_in_spread = (i - len(template.start.split(','))) % len(template.value_spread)
            which_dim = (i - len(template.start.split(','))) // len(template.value_spread)
            which_dim = int(which_dim)
            if value_spread[which_cell_in_spread] == 'value':
                row[i] = dims[which_dim]
    return row
            
            
def LDUPgenerate_header_row(headernames):
    header_row = [(k[0] if isinstance(k, tuple) else k)  for k in template.headermeasurements]

    # create new header row
    for i in range(len(headernames)-1):
        header_row.extend(template.repeat.format(num=i+1).split(','))

    # overwrite dimensions/subject/name as column header (if requested)
    if template.topic_headers_as_dims:
        dims = []
        for headername in headernames:
            dims.append(headername)
        header_row = Lrewrite_headers(header_row, dims)
    return header_row


def LwritetechnicalCSV(outputfile, conversionsegments):
    if type(conversionsegments) is ConversionSegment:
        conversionsegments = [conversionsegments]
        
    if outputfile is not None:
        print("writing %d conversion segments into %s" % (len(conversionsegments), os.path.abspath(outputfile)))
        filehandle = open(outputfile, "w", newline='\n')
    else:
        filehandle = io.StringIO()
    csv_writer = csv.writer(filehandle)
    row_count = 0
        
    for i, conversionsegment in enumerate(conversionsegments):
        headernames = [None]+[dimension.label  for dimension in conversionsegment.dimensions  if type(dimension.label) != int ]
        if i == 0:   # only first segment
            header_row = LDUPgenerate_header_row(headernames)
            csv_writer.writerow(header_row)
            
        if conversionsegment.processedrows is None: 
            conversionsegment.process()  
            
        kdim = dict((dimension.label, dimension)  for dimension in conversionsegment.dimensions)
        timeunitmessage = ""
        if template.SH_Create_ONS_time and ((template.TIMEUNIT not in kdim) and (template.TIME in kdim)):
            timeunitmessage = conversionsegment.guesstimeunit()
            conversionsegment.fixtimefromtimeunit()
        elif template.TIME in kdim and template.TIMEUNIT not in kdim:
            conversionsegment.fixtimefromtimeunit()

        if outputfile is not None:
            print("conversionwrite segment size %d table '%s; %s" % (len(conversionsegment.processedrows), conversionsegment.tab.name, timeunitmessage))
        for row in conversionsegment.processedrows:
            values = dict((k if type(k)==int else headernames.index(k), v)  for k, v in row.items())
            output_row = Lyield_dimension_values(values, headernames)
            csv_writer.writerow(output_row)
            row_count += 1
            
    csv_writer.writerow(["*"*9, row_count])
    if outputfile is not None:
        filehandle.close()
    else:
        return filehandle.getvalue()


    
    
    

In [13]:
x, Lx = writetechnicalCSV(None, c1), LwritetechnicalCSV(None, c1)
x == Lx, len(x), len(Lx)

(True, 2744, 2744)

In [8]:
Lx

'observation,data_marking,statistical_unit_eng,statistical_unit_cym,measure_type_eng,measure_type_cym,observation_type,empty,obs_type_value,unit_multiplier,unit_of_measure_eng,unit_of_measure_cym,confidentuality,empty1,geographic_area,empty2,empty3,time_dim_item_id,time_dim_item_label_eng,time_dim_item_label_cym,time_type,empty4,statistical_population_id,statistical_population_label_eng,statistical_population_label_cym,cdid,cdiddescrip,empty5,empty6,empty7,empty8,empty9,empty10,empty11,empty12,dim_id_1,dimension_label_eng_1,dimension_label_cym_1,dim_item_id_1,dimension_item_label_eng_1,dimension_item_label_cym_1,is_total_1,is_sub_total_1,dim_id_2,dimension_label_eng_2,dimension_label_cym_2,dim_item_id_2,dimension_item_label_eng_2,dimension_item_label_cym_2,is_total_2,is_sub_total_2,dim_id_3,dimension_label_eng_3,dimension_label_cym_3,dim_item_id_3,dimension_item_label_eng_3,dimension_item_label_cym_3,is_total_3,is_sub_total_3\r\n2.0,,,,,,,,,,,,,,,,,2014,2014,,Year,,,,,,,,,,,,,,,Vehicle

In [11]:
six.text_type(666)

'666'

In [13]:
print(Lx)

observation,data_marking,statistical_unit_eng,statistical_unit_cym,measure_type_eng,measure_type_cym,observation_type,empty,obs_type_value,unit_multiplier,unit_of_measure_eng,unit_of_measure_cym,confidentuality,empty1,geographic_area,empty2,empty3,time_dim_item_id,time_dim_item_label_eng,time_dim_item_label_cym,time_type,empty4,statistical_population_id,statistical_population_label_eng,statistical_population_label_cym,cdid,cdiddescrip,empty5,empty6,empty7,empty8,empty9,empty10,empty11,empty12,dim_id_1,dimension_label_eng_1,dimension_label_cym_1,dim_item_id_1,dimension_item_label_eng_1,dimension_item_label_cym_1,is_total_1,is_sub_total_1,dim_id_2,dimension_label_eng_2,dimension_label_cym_2,dim_item_id_2,dimension_item_label_eng_2,dimension_item_label_cym_2,is_total_2,is_sub_total_2,dim_id_3,dimension_label_eng_3,dimension_label_cym_3,dim_item_id_3,dimension_item_label_eng_3,dimension_item_label_cym_3,is_total_3,is_sub_total_3
2.0,,,,,,,,,,,,,,,,,2014,2014,,Year,,,,,,,,,,,,,,,Vehicles,Ve

In [63]:

import collections
headermeasurements = [
    ('observation',            "OBS"), 
    ('data_marking',           "DATAMARKER"), 
    ('statistical_unit_eng',   "STATUNIT"),      'statistical_unit_cym', 
    ('measure_type_eng',       "MEASURETYPE"),   'measure_type_cym', 'observation_type', 'empty', 'obs_type_value', 
    ('unit_multiplier',        "UNITMULTIPLIER"),
    ('unit_of_measure_eng',    "UNITOFMEASURE"), 'unit_of_measure_cym', 'confidentuality', 'empty1', 
    ('geographic_area',        "GEOG"),          'empty2', 'empty3', 
    ('time_dim_item_id',       "TIME"),               
    ('time_dim_item_label_eng',"TIME"),          'time_dim_item_label_cym', 
    ('time_type',              "TIMEUNIT"),      'empty4', 
    ('statistical_population_id',       "STATPOP"),
    ('statistical_population_label_eng',"STATPOP"),
            'statistical_population_label_cym', 'cdid', 'cdiddescrip', 'empty5', 'empty6', 'empty7', 'empty8', 'empty9', 'empty10', 'empty11', 'empty12'
]

# derive the elements of the headernames above into the values below
start = ",".join((k[0] if isinstance(k, tuple) else k) for k in headermeasurements)
headermeasurementnames = list(collections.OrderedDict.fromkeys(k[1]  for k in headermeasurements  if isinstance(k, tuple)))
headermeasurementnumvalues = dict((item, -i)  for i, item in enumerate(reversed(dimension_names)))
exec("%s = %s" % (", ".join(headermeasurementnumvalues.keys()), ", ".join(map(str, headermeasurementnumvalues.values()))))

y = [ ]
for k in headermeasurements:
    if isinstance(k, tuple):
        v = values.get(headermeasurementnumvalues[k[1]], '')
        y.append(v)
    else:
        y.append('')
print(y)
len(y)


[999, '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 2001, 2001, '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


35

In [82]:
import collections
headermeasurements = [
    ('observation',            "OBS"), 
    ('data_marking',           "DATAMARKER"), 
    ('statistical_unit_eng',   "STATUNIT"),      'statistical_unit_cym', 
    ('measure_type_eng',       "MEASURETYPE"),   'measure_type_cym', 'observation_type', 'empty', 'obs_type_value', 
    ('unit_multiplier',        "UNITMULTIPLIER"),
    ('unit_of_measure_eng',    "UNITOFMEASURE"), 'unit_of_measure_cym', 'confidentuality', 'empty1', 
    ('geographic_area',        "GEOG"),          'empty2', 'empty3', 
    ('time_dim_item_id',       "TIME"),               
    ('time_dim_item_label_eng',"TIME"),          'time_dim_item_label_cym', 
    ('time_type',              "TIMEUNIT"),      'empty4', 
    ('statistical_population_id',       "STATPOP"),
    ('statistical_population_label_eng',"STATPOP"),
            'statistical_population_label_cym', 'cdid', 'cdiddescrip', 'empty5', 'empty6', 'empty7', 'empty8', 'empty9', 'empty10', 'empty11', 'empty12'
]

# derive the elements of the headernames above into the values below
start = ",".join((k[0] if isinstance(k, tuple) else k) for k in headermeasurements)
headermeasurementnames = list(collections.OrderedDict.fromkeys(k[1]  for k in headermeasurements  if isinstance(k, tuple)))
headermeasurementnumvalues = dict((item, -i)  for i, item in enumerate(reversed(dimension_names)))
exec("%s = %s" % (", ".join(headermeasurementnumvalues.keys()), ", ".join(map(str, headermeasurementnumvalues.values()))))


# used in the old code
DSKIP_AFTER = {OBS: 0,            # 1..2  MANDATORY, must be here, must be called OBS
              DATAMARKER: 0,     # 2..3
              STATUNIT: 1,       # 3..5
              MEASURETYPE: 4,    # 5..10
              UNITMULTIPLIER: 0, # 10..11
              UNITOFMEASURE: 3,  # 11..15
              GEOG: 2,           # 15..18/19
              TIME: 1,           # 18/19..21
              TIMEUNIT: 1,       # 21..23/24
              STATPOP:11}        # 23/24..36/37


headermeasurementnames = list(collections.OrderedDict.fromkeys(k[1]  for k in headermeasurements  if isinstance(k, tuple)))



({-9: 0, -8: 0, -7: 1, -6: 4, -5: 0, -4: 3, -3: 2, -2: 1, -1: 1, 0: 11},
 {-9: 0, -8: 0, -7: 1, -6: 4, -5: 0, -4: 3, -3: 2, -2: 1, -1: 1, 0: 11})